### Imports

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.fumctions import current_timestamp
import os
import sys
import json
# import pandas as pd
# import pyspark
import secret_scope

dir_path = os.getcwd()
root_path = os.path.abspath(os.path.join(dir_path, '..'))
sys.path.append(root_path)

from migration_library.main import tbl_parsed_dict, relation_fct_dict, relation_dim_dict, tbl_name_reformat

### ADLS Connection

In [ ]:
storage_account = secret_scope.storage_account
container_name = secret_scope.container_name
path = secret_scope.path

In [ ]:
spark = SparkSession.builder \
    .appName("sparkwriteapp") \
    .getOrCreate()

# Configure direct access via abfss
spark.conf.set(f"fs.azure.account.auth.type.{storage_account}.dfs.core.windows.net", "ManagedIdentity")         # Assuming IAM is setup and configured
spark.conf.set("fs.azure.account.oauth.msi.endpoint", "http://169.254.169.254/metadata/identity/oauth2/token")

### Write to raw layer
1. Create table in UC
2. With data in external location
3. Save metadata as json file

In [ ]:
# External table write location
adls_path = f"abfss://{container_name}@{storage_account}.dfs.core.windows.net/{path}"

# Catalog and schema information | Can be parameterised in actual usecase
catalog_name = 'data_foundation_dev'
schema_name = 'raw'

In [ ]:
# Write tables to UC with data location specified
for tbl, data in tbl_parsed_dict:
    df = data['df']
    tbl_name = tbl_name_reformat(tbl)

    spark_df = spark.createDataFrame(df)
    storage_path = f"{adls_path}/{schema_name}/{tbl_name}"

    spark_df = spark_df.withColumn('_modified_date', current_timestamp())
    spark_df.write.format(format) \
            .mode("overwrite") \
            .option("overwriteSchema", "true") \
            .option("path", adls_path) \
            .saveAsTable(f"{catalog_name}.{schema_name}.{tbl_name}")
    
    print(f"External table `{catalog_name}`.{schema_name}.{tbl_name} created successfully with data at {storage_path}.")

In [ ]:
# Enforce keys

for tbl, data in tbl_parsed_dict:
    tbl_name = tbl_name_reformat(tbl)
    key = data['pk']

    enforce_pk = spark.sql(f"""ALTER TABLE {catalog_name}.{schema_name}.{tbl_name}
                           ADD PRIMARY KEY ({key}) RELY;""")
    print(f"PK enforced for `{catalog_name}`.{schema_name}.{tbl_name}")
    
for tbl, relation in relation_fct_dict:
    tbl_name = tbl_name_reformat(tbl)

    for key, nxt_tbl in relation:
        enforce_fk = spark.sql(f"""ALTER TABLE {catalog_name}.{schema_name}.{tbl_name}
                            ADD FOREIGN KEY ({key}) REFERENCES {catalog_name}.{schema_name}.{nxt_tbl}({key});""")

        print(f"FK enforced for `{catalog_name}`.{schema_name}.{tbl_name}")

In [ ]:
# Save table metadata in workspace or volume for downstream use

metadata_path = '..\\output_obj\\tbl_metadata.json'

tbl_metadata = {}
for tbl, data in tbl_parsed_dict.items():
    tbl_name = tbl_name_reformat(tbl)
    tbl_metadata[tbl_name] = {i:j for i,j in data.items() if i != 'df'}

    try:
        tbl_metadata[tbl_name]['fk'] = {i:tbl_name_reformat(j) for i,j in relation_fct_dict[tbl]}
    except:
        tbl_metadata[tbl_name]['fk'] = {}
        # tbl_metadata[tbl]['fk'] = {i:j for i,j in relation_dim_dict[tbl]}

with open(metadata_path, "w") as f:
    json.dump(tbl_metadata, f, indent=2)

print(f"Metadata saved at {metadata_path}")